In [9]:
import os
import json
import pandas as pd
import requests
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100
pd.set_option('display.max_colwidth', -1)

/var/folders/st/8jn3gyls0_nbxy23bw4zmds00000gn/T/ipykernel_66512/2167127486.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [32]:
def getDataframe(filename):
    arr = []
    with open('./data/' + filename) as out:
        for line in out:
            j_content = json.loads(line)
            arr.append(j_content)
    df = pd.DataFrame(arr)
    return df

In [33]:
def mongoIdToId(value):
    return dict(value)["$oid"]

In [34]:
producers = getDataframe('producers.json')
producers["organization_id"] = producers["associatedTo"].apply(mongoIdToId)
producers.head()

,_id,associatedTo,__v,organization_id
0,AFB,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123
1,AFB - DR Nouvelle-Aquitaine,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123
2,ALTOA,{'$oid': '580fb83a88ee38771813e4cb'},NaN,580fb83a88ee38771813e4cb
3,ANFR,{'$oid': '54256fe588ee380af6717348'},NaN,54256fe588ee380af6717348
4,ARS,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123


In [35]:
organizations = getDataframe('organizations.json')
organizations["geop_organization_id"] = organizations["_id"].apply(mongoIdToId)
organizations = organizations.merge(producers, left_on="geop_organization_id", right_on="organization_id")
organizations["geop_organization_name"] = organizations["_id_y"]
organizations = organizations[["geop_organization_id", "geop_organization_name"]]
organizations.head()

,geop_organization_id,geop_organization_name
0,534fff68a3a7292c64a77d8d,Département de Saône-et-Loire
1,534fff68a3a7292c64a77d8d,Département de Saône-et-Loire (71)
2,534fff6aa3a7292c64a77d96,Région Hauts-de-France
3,534fff6ba3a7292c64a77da1,ASPA
4,534fff6ba3a7292c64a77da1,CConseil Départemental du Bas-Rhin (CD67)


In [36]:
organizations['dgv_organization_id'] = organizations['geop_organization_id']

In [37]:
dgv_catalog_orgas = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/b7bbfedc-2448-4135-a6c7-104548d396e7',sep=';')

In [38]:
dgv_catalog_orgas = dgv_catalog_orgas[['id','name', 'metric.datasets']]
dgv_catalog_orgas.head()

,id,name,metric.datasets
0,6205247a142c650419d4733e,Saint-Julia,0
1,6204e8275f491f6083e9e3ea,Société d'Enlèvement et de NETtoyage,0
2,6204e75f281e0f8644faf936,Société d'Enlèvement et de NETtoyage,0
3,6204e6ac5f491f6083e9e3e9,Société d'Enlèvement et de NETtoyage,0
4,6204e66aea1bca8b549b16eb,SNC PERFOTEL OCEANIA ST GREGOIRE,0


In [39]:
dgv_catalog_orgas = dgv_catalog_orgas.rename(columns={
    'id': 'dgv_organization_id',
    'name': 'dgv_organization_name', 
    'metric.datasets': 'dgv_nb_datasets',
})

In [40]:
# no duplicates
assert organizations[organizations.duplicated()].empty
assert dgv_catalog_orgas[dgv_catalog_orgas.duplicated()].empty

In [41]:
organizations.columns

Index(['geop_organization_id', 'geop_organization_name',
       'dgv_organization_id'],
      dtype='object')

In [42]:
dgv_catalog_orgas.dgv_organization_id.nunique()

3239

In [43]:
# outer merge to pivot on dgv orgas, not necessarily on geop orgs
final_orgas = dgv_catalog_orgas.merge(organizations, on='dgv_organization_id', how='outer')
assert final_orgas[final_orgas.duplicated()].empty
final_orgas.head()

,dgv_organization_id,dgv_organization_name,dgv_nb_datasets,geop_organization_id,geop_organization_name
0,6205247a142c650419d4733e,Saint-Julia,0.0,NaN,NaN
1,6204e8275f491f6083e9e3ea,Société d'Enlèvement et de NETtoyage,0.0,NaN,NaN
2,6204e75f281e0f8644faf936,Société d'Enlèvement et de NETtoyage,0.0,NaN,NaN
3,6204e6ac5f491f6083e9e3e9,Société d'Enlèvement et de NETtoyage,0.0,NaN,NaN
4,6204e66aea1bca8b549b16eb,SNC PERFOTEL OCEANIA ST GREGOIRE,0.0,NaN,NaN


In [44]:
geoc = pd.read_excel('./data/Liste_catalogues_DCAT.xlsx')

In [45]:
geoc = geoc.rename(columns={
    'Producteurs': 'geoc_organization_name', 
    'uuid': 'geoc_organization_uuid', 
    'url de moissonnage ': 'geoc_organization_url'
})
geoc.head()

,geoc_organization_name,geoc_organization_uuid,geoc_organization_url
0,Académie de Créteil,b0af64e401fc6549f263001cfc288db5,http://www.geocatalogue.fr/www/dcat/b0af64e401fc6549f263001cfc288db5/catalog.rdf
1,Académie de Montpellier,5be9cd826e5938bb5e8dd269bf57180d,http://www.geocatalogue.fr/www/dcat/5be9cd826e5938bb5e8dd269bf57180d/catalog.rdf
2,Administration centrale MEEM-MLHD (MEEM),e5982bd5379515aecfa45fa3abea539b,http://www.geocatalogue.fr/www/dcat/e5982bd5379515aecfa45fa3abea539b/catalog.rdf
3,Aerodata,8ec73ccd0e37440d31827b7445c2d050,http://www.geocatalogue.fr/www/dcat/8ec73ccd0e37440d31827b7445c2d050/catalog.rdf
4,Aero Photo Europe Investigation (APEI),5cea64971edfdaf4effe0f33d834e896,http://www.geocatalogue.fr/www/dcat/5cea64971edfdaf4effe0f33d834e896/catalog.rdf


In [46]:
def removeCommonWords(org):
    org = str(org)
    org = org.replace(' de ',' ').replace(' un ',' ').replace(' à ','').replace(' l\'',' ').replace(' d\'', ' ')
    org = org.replace(' des ',' ').replace(' pour ', ' ').replace(' les ', ' ').replace('(','').replace(')','')
    org = org.replace(' - ',' ').replace(',','').replace(' du ', ' ').replace('à','a').replace('é','e')
    org = org.replace('ê','e').replace('è','e').replace('\'',' ')
    return org.lower()

In [47]:
geoc["geoc_slug"] = geoc["geoc_organization_name"].apply(removeCommonWords)
final_orgas["geop_slug"] = final_orgas["geop_organization_name"].apply(removeCommonWords)
final_orgas["dgv_slug"] = final_orgas["dgv_organization_name"].apply(removeCommonWords)

In [48]:
df_dgv_match = final_orgas.merge(geoc, left_on="dgv_slug", right_on="geoc_slug")
df_dgv_match["geoc_organization_url"].drop_duplicates().shape

(109,)

In [49]:
df_geop_match = final_orgas.merge(geoc, left_on="geop_slug", right_on="geoc_slug")
df_geop_match["geoc_organization_url"].drop_duplicates().shape

(36,)

In [50]:
df_final = df_dgv_match.merge(df_geop_match, how="outer")
df_final[df_final['geoc_organization_uuid'].notna()]["geoc_organization_url"].drop_duplicates().shape

(132,)

In [51]:
df_final[df_final['geoc_organization_uuid'].notna()].head()

,dgv_organization_id,dgv_organization_name,dgv_nb_datasets,geop_organization_id,geop_organization_name,geop_slug,dgv_slug,geoc_organization_name,geoc_organization_uuid,geoc_organization_url,geoc_slug
0,61fc10b5285e11c883d018bd,MAIRIE LES TROIS DOMAINES,0.0,NaN,NaN,nan,mairie les trois domaines,Mairie de Les Trois domaines,6fc21c45008161fcadd99a9c53e0351f,http://www.geocatalogue.fr/www/dcat/6fc21c45008161fcadd99a9c53e0351f/catalog.rdf,mairie les trois domaines
1,61b0b0673c11d387da1df057,Pontivy Communaute,0.0,NaN,NaN,nan,pontivy communaute,Pontivy Communauté,eac49cc9bdb321a31f6150862adc28b7,http://www.geocatalogue.fr/www/dcat/eac49cc9bdb321a31f6150862adc28b7/catalog.rdf,pontivy communaute
2,61484aaaffc949c8b4971434,Préfecture de la région Hauts-de-France,1.0,NaN,NaN,nan,prefecture la region hauts-de-france,Préfecture de la région Hauts-de-France,0709d43ca86491e10b8d95b9ae7ad916,http://www.geocatalogue.fr/www/dcat/0709d43ca86491e10b8d95b9ae7ad916/catalog.rdf,prefecture la region hauts-de-france
3,61322119e8f2ff4e8225c747,Préfecture des Landes,0.0,NaN,NaN,nan,prefecture landes,Préfecture des Landes,81bbfc2b3e0c3589fb5f868f157f0f65,http://www.geocatalogue.fr/www/dcat/81bbfc2b3e0c3589fb5f868f157f0f65/catalog.rdf,prefecture landes
4,60ba2f3eb3735a080f1fbda7,Communauté de Communes de Haute Cornouaille,15.0,NaN,NaN,nan,communaute communes haute cornouaille,Communauté de Communes de Haute Cornouaille,1f9c319f589bbea6f8bbf65bcf6be730,http://www.geocatalogue.fr/www/dcat/1f9c319f589bbea6f8bbf65bcf6be730/catalog.rdf,communaute communes haute cornouaille


In [69]:
df_final.to_csv('mapping_geop_dgv_geoc-pivot-datagouv.csv',index=False)

In [64]:
df_final.shape

(188, 11)

In [60]:
from collections import defaultdict

url = "https://www.data.gouv.fr/api/1"
auth = {
    "x-api-key": os.getenv("DATAGOUV_TOKEN")
}

def get_org_data(org_id):
    org_data = defaultdict(list)
    org_data["id"] = org_id
    r = requests.get(f"{url}/organizations/{org_id}/")
    data = r.json()

    # members
    for m in data["members"]:
        r = requests.get(m["user"]["uri"], headers=auth)
        mdata = r.json()
        org_data[m["role"]].append(f"{mdata['first_name']} {mdata['last_name']} <{mdata['email']}>")
    
    org_data["admin"] = ", ".join(org_data["admin"])
    org_data["editor"] = ", ".join(org_data["editor"])

    # datasets
    r = requests.get(f"{url}/organizations/{org_id}/datasets/?page_size=1000")
    datasets = r.json()
    assert datasets["total"] == len(datasets["data"])
    org_data["nb_datasets"] = datasets["total"]
    org_data["nb_datasets_geop"] = len([d["id"] for d in datasets["data"] if "geop:dataset_id" in d["extras"]])
    org_data["last_dataset_modified_date"] = max([d["last_modified"] for d in datasets["data"]]) if datasets["data"] else None
        
    return dict(org_data)

In [61]:
metadata = []
for org_id in df_final["dgv_organization_id"].drop_duplicates():
    metadata.append(get_org_data(org_id))

In [ ]:
del df_final["dgv_nb_datasets"]

In [70]:
meta_df = pd.DataFrame(metadata)
df_meta = df_final.merge(meta_df, right_on="id", left_on="dgv_organization_id")

In [71]:
df_meta.to_csv('mapping_geop_dgv_geoc-pivot-datagouv_w_metadata.csv',index=False)

In [26]:
# TODO:
# - [x] add nb of datasets
# - [x] add nb of geop datasets
# - [x] add admin contacts
# - [x] add members contacts
# - [x] last modified dataset date
# - [x] match w/o joining on geop_organizations (dgv_organizations)